This notebook is for taking our transformer models and applying SHAP explanations to their output to see which tokens contribute to what.

In [61]:
from torch.utils.data import DataLoader
from dataset.phishing_dataset import PhishingDataset

from classifiers.basic_url_skinny_bert_classifier import BasicUrlSkinnyBertClassifier
import torch
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
import shap
import scipy as sp
import numpy as np
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model = BasicUrlSkinnyBertClassifier()
checkpoint_path = "../models/canonical/basic_url_skinny_bert_phishing_classifier_epoch_1.pt"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.eval()

dataset_path = os.path.expanduser("~/transfer/phishing_output_tokenized.h5")
test_dataset = PhishingDataset(['url'], split='dev', local_file_path=dataset_path)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=True)

# Define a wrapper for SHAP compatibility
class BertClassifierWrapper(nn.Module):
    def __init__(self, model):
        super(BertClassifierWrapper, self).__init__()
        self.model = model

    def forward(self, tokenized_inputs):
        input_ids = tokenized_inputs.to(device)
        attention_mask = (input_ids != 0).long().to(device)
        return self.model(input_ids, attention_mask)

wrapped_model = BertClassifierWrapper(model)

feature_names = tokenizer.convert_ids_to_tokens(range(len(tokenizer)))

# prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, max_length=128, padding='max_length', truncation=True) for v in x])
    class_preds = wrapped_model(tv)
    outputs = class_preds.detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])
    return val

batch = test_dataloader.__iter__().__next__()
explainer = shap.Explainer(f, tokenizer)
urls = batch['url']
shap_values = explainer(urls, fixed_context=1)



/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/var/folders/j1/dctzbvps2w7cgcp3cflmcxm00000gn/T/ipykernel_76972/2025486221.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are ex

In [69]:
file = open('temp.html','w')
file.write(shap.plots.text(shap_values[:6], display=False))
file.close

TypeError: text() got an unexpected keyword argument 'show'